In [ ]:
#Demo to show case stable diffusion using determined

## Before You Start: 🤗 Account, Access Token, and License

In order to use this repository's implementation of Stable Diffusion, you must:

* Have a [Huggingface account](https://huggingface.co/join).
* Have a [Huggingface User Access Token](https://huggingface.co/docs/hub/security-tokens).
* Accept the Stable Diffusion license (click on _Access
  repository_  [in this link)](https://huggingface.co/CompVis/stable-diffusion-v1-4).


## Training Setup

1. After including your user access token in the `const.yaml` config file by modifying the final part
of the lines which read

2. Update the resource pool name that has the gpu's you are assigned

```yaml
environment:
  environment_variables:
    - HF_AUTH_TOKEN=YOUR_HF_AUTH_TOKEN_HERE
```

## Fine-tuning
Fine-tuning experiment can be run by executing the following in the present directory

The below command will submit an experiment which introduces a new embedding vector into the world of Stable
Diffusion which we will train to correspond to the concept of the Determined AI logo, as represented
through
training images found in `/det_logos`


In [ ]:
!det experiment create const.yaml .

By default, sample images are generated during training which can be viewed by launching a
Tensorboard instance from the experiment in the WebUI.

## Prompt Inference